In [1]:
import pandas as pd
import numpy as np

mimicKG = pd.read_csv('mid\mimicKG.csv')
sdoh = pd.read_csv('C:\data\RA\shenlab\SDoH&KG\data\SDoHKG_matched.csv')
pkg = pd.read_csv(r'C:\data\RA\shenlab\SDoH&KG\data\kg.csv')

C:\Users\Tianqi\AppData\Local\Temp\ipykernel_22684\1351530523.py:6: DtypeWarning: Columns (3,8) have mixed types. Specify dtype option on import or set low_memory=False.
  pkg = pd.read_csv(r'C:\data\RA\shenlab\SDoH&KG\data\kg.csv')


In [2]:
mimicKG

,head,relation,tail,weight,head_type,tail_type
0,*NF* Bicalutamide,drug_sdoh,admission,16,drug,SDoH
1,*NF* Bicalutamide,drug_sdoh,appointment,16,drug,SDoH
2,*NF* Bicalutamide,drug_sdoh,aricept 10 mg,16,drug,SDoH
3,*NF* Bicalutamide,drug_sdoh,assisted care facility,16,drug,SDoH
4,*NF* Bicalutamide,drug_sdoh,bicalutamide,16,drug,SDoH
...,...,...,...,...,...,...
923384,widowed,marital_status_drug,Vitamin D,60,marital_status,drug
923385,widowed,marital_status_drug,Warfarin,41,marital_status,drug
923386,widowed,marital_status_drug,Zolpidem Tartrate,12,marital_status,drug
923387,widowed,marital_status_drug,traZODONE,18,marital_status,drug


In [5]:
# number of rows of each relation
print(mimicKG.groupby('relation').size())

relation
age_disease                  140
age_drug                     540
disease_sdoh              308524
drug_disease              183927
drug_sdoh                 426783
ethnicity_disease            215
ethnicity_drug               409
gender_disease               246
gender_drug                  449
insurance_disease            219
insurance_drug               353
marital_status_disease       231
marital_status_drug          547
religion_disease             237
religion_drug                569
dtype: int64


In [6]:
# get unique x_type
print(sdoh['x_type'].unique())

['Disease' 'Species' 'Chemical' 'Gene' 'Variant' 'CellLine' 'SDoH factor'
 'SDoH subtype']


In [12]:
# if x_type is 'Disease', get unique x_id to a list
disease = sdoh[sdoh['x_type'] == 'Disease']
disease = disease['x_id'].unique().tolist()
chemical = sdoh[sdoh['x_type'] == 'Chemical']
chemical = chemical['x_id'].unique().tolist()
# remove 'MESH:' from the beginning of each element in the list
disease = [x.replace('MESH:', '') for x in disease]
chemical = [x.replace('MESH:', '') for x in chemical]
# save the list to 2 txt files
with open('sdoh_disease.txt', 'w') as f:
    for item in disease:
        f.write("%s\n" % item)
with open('sdoh_drug.txt', 'w') as f:
    for item in chemical:
        f.write("%s\n" % item)

In [27]:
sdoh1 = sdoh[sdoh['y_type'] == 'sdoh factor']
sdoh1 = sdoh1['y_name'].unique().tolist()


In [29]:
with open('sdoh_sdoh.txt', 'w') as f:
    for item in sdoh1:
        f.write("%s\n" % item)

In [4]:
sdoh = sdoh.drop(columns=['x_index', 'x_id', 'x_source', 'y_index', 'y_id', 'y_source', 'pmid'])
# drop all the rows with 'relation'='subordinate'
sdoh = sdoh[sdoh['relation'] != 'subordinate'] 
sdoh

,relation,x_name,x_type,y_name,y_type,probability
0,associates,dementia (disease),disease,diagnosis,SDoH factor,0.5
1,associates,dementia (disease),disease,disease-modifying treatments,SDoH factor,0.5
2,associates,dementia (disease),disease,elderly care,SDoH factor,0.5
3,associates,dementia (disease),disease,guidelines,SDoH factor,0.5
4,associates,dementia (disease),disease,protocols,SDoH factor,0.5
...,...,...,...,...,...,...
2704,associates,heart disease,disease,poor nutrition,SDoH factor,0.5
2705,associates,respiratory system disease,disease,poor nutrition,SDoH factor,1.0
2706,associates,dementia (disease),disease,disparities,SDoH factor,1.0
2707,associates,dementia (disease),disease,representation,SDoH factor,0.5


In [5]:
# lowercase everything in 'x_type' and 'y_type'
sdoh['x_type'] = sdoh['x_type'].str.lower()
sdoh['y_type'] = sdoh['y_type'].str.lower()
# if x_type = chemical, change it to drug
sdoh.loc[sdoh['x_type'] == 'chemical', 'x_type'] = 'drug'
sdoh

,relation,x_name,x_type,y_name,y_type,probability
0,associates,dementia (disease),disease,diagnosis,sdoh factor,0.5
1,associates,dementia (disease),disease,disease-modifying treatments,sdoh factor,0.5
2,associates,dementia (disease),disease,elderly care,sdoh factor,0.5
3,associates,dementia (disease),disease,guidelines,sdoh factor,0.5
4,associates,dementia (disease),disease,protocols,sdoh factor,0.5
...,...,...,...,...,...,...
2704,associates,heart disease,disease,poor nutrition,sdoh factor,0.5
2705,associates,respiratory system disease,disease,poor nutrition,sdoh factor,1.0
2706,associates,dementia (disease),disease,disparities,sdoh factor,1.0
2707,associates,dementia (disease),disease,representation,sdoh factor,0.5


In [30]:
utype = sdoh['x_type'].unique()
utype

array(['disease', 'species', 'drug', 'gene', 'variant', 'cellline'],
      dtype=object)

In [31]:
pkg

,relation,display_relation,x_index,x_id,x_type,x_name,x_source,y_index,y_id,y_type,y_name,y_source
0,protein_protein,ppi,0,9796,gene/protein,PHYHIP,NCBI,8889,56992,gene/protein,KIF15,NCBI
1,protein_protein,ppi,1,7918,gene/protein,GPANK1,NCBI,2798,9240,gene/protein,PNMA1,NCBI
2,protein_protein,ppi,2,8233,gene/protein,ZRSR2,NCBI,5646,23548,gene/protein,TTC33,NCBI
3,protein_protein,ppi,3,4899,gene/protein,NRF1,NCBI,11592,11253,gene/protein,MAN1B1,NCBI
4,protein_protein,ppi,4,5297,gene/protein,PI4KA,NCBI,2122,8601,gene/protein,RGS20,NCBI
...,...,...,...,...,...,...,...,...,...,...,...,...
8100493,anatomy_protein_absent,expression absent,66747,4720,anatomy,cerebellar vermis,UBERON,5259,140,gene/protein,ADORA3,NCBI
8100494,anatomy_protein_absent,expression absent,63824,1377,anatomy,quadriceps femoris,UBERON,58254,105378952,gene/protein,KLF18,NCBI
8100495,anatomy_protein_absent,expression absent,63826,1379,anatomy,vastus lateralis,UBERON,58254,105378952,gene/protein,KLF18,NCBI
8100496,anatomy_protein_absent,expression absent,64523,2084,anatomy,heart left ventricle,UBERON,58254,105378952,gene/protein,KLF18,NCBI


In [32]:
unique_x = pkg['x_type'].unique()
unique_y = pkg['y_type'].unique()
unique_x, unique_y

(array(['gene/protein', 'drug', 'effect/phenotype', 'disease',
        'biological_process', 'molecular_function', 'cellular_component',
        'exposure', 'pathway', 'anatomy'], dtype=object),
 array(['gene/protein', 'disease', 'drug', 'effect/phenotype',
        'biological_process', 'molecular_function', 'cellular_component',
        'exposure', 'pathway', 'anatomy'], dtype=object))

In [37]:
disease = pkg[(pkg['x_type'] == 'disease') | (pkg['y_type'] == 'disease')]
disease

,relation,display_relation,x_index,x_id,x_type,x_name,x_source,y_index,y_id,y_type,y_name,y_source
346728,contraindication,contraindication,15193,DB05271,drug,Rotigotine,DrugBank,33577,5044,disease,hypertensive disorder,MONDO
346729,contraindication,contraindication,15193,DB05271,drug,Rotigotine,DrugBank,36035,1200_1134_15512_5080_100078,disease,hypertension,MONDO_grouped
346730,indication,indication,16687,DB00492,drug,Fosinopril,DrugBank,33577,5044,disease,hypertensive disorder,MONDO
346731,indication,indication,16687,DB00492,drug,Fosinopril,DrugBank,36035,1200_1134_15512_5080_100078,disease,hypertension,MONDO_grouped
346732,contraindication,contraindication,14483,DB13956,drug,Estradiol valerate,DrugBank,33577,5044,disease,hypertensive disorder,MONDO
...,...,...,...,...,...,...,...,...,...,...,...,...
6499510,exposure_disease,linked to,31393,11565,disease,metabolic syndrome X,MONDO,61825,D015032,exposure,Zinc,CTD
6499511,exposure_disease,linked to,28208,5148,disease,type 2 diabetes mellitus,MONDO,61825,D015032,exposure,Zinc,CTD
6499512,exposure_disease,linked to,84172,1076,disease,glucose intolerance,MONDO,61825,D015032,exposure,Zinc,CTD
6499513,exposure_disease,linked to,33063,100130,disease,adult acute respiratory distress syndrome,MONDO,61825,D015032,exposure,Zinc,CTD


In [41]:
disease1 = pkg[pkg['x_type'] == 'disease']
disease2 = pkg[pkg['y_type'] == 'disease']
disease1 = disease1['x_name'].unique().tolist()
disease2 = disease2['y_name'].unique().tolist()
# put disease1 and 2 together and deduplicate
disease = list(set(disease1 + disease2))
with open('mapping/pkg_disease.txt', 'w') as f:
    for item in disease:
        f.write("%s\n" % item)

In [52]:
drug1 = pkg[pkg['x_type'] == 'drug']
drug2 = pkg[pkg['y_type'] == 'drug']
drug1 = drug1['x_name'].unique().tolist()
drug2 = drug2['y_name'].unique().tolist()
# put drug1 and 2 together and deduplicate
drug = list(set(drug1 + drug2))
with open('mapping/pkg_drug1.txt', 'w') as f:
    for item in drug:
        f.write("%s\n" % item)

In [7]:
pkg = pkg.drop(columns=['display_relation', 'x_index', 'x_id', 'x_source', 'y_index', 'y_id', 'y_source'])
pkg

,relation,x_type,x_name,y_type,y_name
0,protein_protein,gene/protein,PHYHIP,gene/protein,KIF15
1,protein_protein,gene/protein,GPANK1,gene/protein,PNMA1
2,protein_protein,gene/protein,ZRSR2,gene/protein,TTC33
3,protein_protein,gene/protein,NRF1,gene/protein,MAN1B1
4,protein_protein,gene/protein,PI4KA,gene/protein,RGS20
...,...,...,...,...,...
8100493,anatomy_protein_absent,anatomy,cerebellar vermis,gene/protein,ADORA3
8100494,anatomy_protein_absent,anatomy,quadriceps femoris,gene/protein,KLF18
8100495,anatomy_protein_absent,anatomy,vastus lateralis,gene/protein,KLF18
8100496,anatomy_protein_absent,anatomy,heart left ventricle,gene/protein,KLF18


In [8]:
relation1 = pkg['relation'].unique()
relation1

array(['protein_protein', 'drug_protein', 'contraindication',
       'indication', 'off-label use', 'drug_drug', 'phenotype_protein',
       'phenotype_phenotype', 'disease_phenotype_negative',
       'disease_phenotype_positive', 'disease_protein', 'disease_disease',
       'drug_effect', 'bioprocess_bioprocess', 'molfunc_molfunc',
       'cellcomp_cellcomp', 'molfunc_protein', 'cellcomp_protein',
       'bioprocess_protein', 'exposure_protein', 'exposure_disease',
       'exposure_exposure', 'exposure_bioprocess', 'exposure_molfunc',
       'exposure_cellcomp', 'pathway_pathway', 'pathway_protein',
       'anatomy_anatomy', 'anatomy_protein_present',
       'anatomy_protein_absent'], dtype=object)

In [55]:
# Read the first text file
with open('mapping\pkg_drug.txt', 'r') as file1:
    data1 = file1.readlines()

# Read the second text file
with open('mapping\pkg_drug_cui1.txt', 'r') as file2:
    data2 = file2.readlines()


# Strip newlines and create a DataFrame with two columns
df = pd.DataFrame({
    'name': [line.strip() for line in data1],
    'cui': [line.strip() for line in data2]
})

# Display the DataFrame
print(df)

# Optionally save the DataFrame to a CSV or Excel file
df.to_csv('pkg_drug.csv', index=False)


         name                cui
0     DB04878           C0020264
1     DB07916           C1871526
2     DB11540           C0301379
3     DB01671  No results found.
4     DB05676           C4535718
...       ...                ...
7952  DB03513           C0067067
7953  DB08259           C0055147
7954  DB08246           C0044494
7955  DB08325           C0076275
7956  DB14627           C0387149

[7957 rows x 2 columns]


In [3]:
mimicKG

,head,relation,tail,weight,head_type,tail_type
0,*NF* Bicalutamide,drug_sdoh,admission,16,drug,SDoH
1,*NF* Bicalutamide,drug_sdoh,appointment,16,drug,SDoH
2,*NF* Bicalutamide,drug_sdoh,aricept 10 mg,16,drug,SDoH
3,*NF* Bicalutamide,drug_sdoh,assisted care facility,16,drug,SDoH
4,*NF* Bicalutamide,drug_sdoh,bicalutamide,16,drug,SDoH
...,...,...,...,...,...,...
923384,widowed,marital_status_drug,Vitamin D,60,marital_status,drug
923385,widowed,marital_status_drug,Warfarin,41,marital_status,drug
923386,widowed,marital_status_drug,Zolpidem Tartrate,12,marital_status,drug
923387,widowed,marital_status_drug,traZODONE,18,marital_status,drug


In [5]:
group1 = mimicKG
df1 = pd.read_csv('mapping/mimic_drug.csv', low_memory=False)
df2 = pd.read_csv('mapping/mimic_disease.csv', low_memory=False)
group1['head_id'] = None
group1['tail_id'] = None
for index, row in group1.iterrows():
    if row['head_type'] == 'drug':
        head = row['head']
        cui = df1[df1['name'] == head]['cui'].values
        if len(cui) > 0:
            group1.at[index, 'head_id'] = cui[0]
    elif row['tail_type'] == 'drug':
        tail = row['tail']
        cui = df1[df1['name'] == head]['cui'].values
        if len(cui) > 0:
            group1.at[index, 'tail_id'] = cui[0]
    elif row['head_type'] == 'disease':
        head = row['head']
        cui = df2[df2['name'] == head]['cui'].values
        if len(cui) > 0:
            group1.at[index, 'head_id'] = cui[0]
    elif row['tail_type'] == 'disease':
        tail = row['tail']
        cui = df2[df2['name'] == head]['cui'].values
        if len(cui) > 0:
            group1.at[index, 'tail_id'] = cui[0]
            
group1

,head,relation,tail,weight,head_type,tail_type,head_id,tail_id
0,*NF* Bicalutamide,drug_sdoh,admission,16,drug,SDoH,C0285590,None
1,*NF* Bicalutamide,drug_sdoh,appointment,16,drug,SDoH,C0285590,None
2,*NF* Bicalutamide,drug_sdoh,aricept 10 mg,16,drug,SDoH,C0285590,None
3,*NF* Bicalutamide,drug_sdoh,assisted care facility,16,drug,SDoH,C0285590,None
4,*NF* Bicalutamide,drug_sdoh,bicalutamide,16,drug,SDoH,C0285590,None
...,...,...,...,...,...,...,...,...
923384,widowed,marital_status_drug,Vitamin D,60,marital_status,drug,None,C0043491
923385,widowed,marital_status_drug,Warfarin,41,marital_status,drug,None,C0043491
923386,widowed,marital_status_drug,Zolpidem Tartrate,12,marital_status,drug,None,C0043491
923387,widowed,marital_status_drug,traZODONE,18,marital_status,drug,None,C0043491


In [7]:
# rename the columns, head_id to head_cui, tail_id to tail_cui
group1 = group1.rename(columns={'head_id': 'head_cui', 'tail_id': 'tail_cui'})
group1

,head,relation,tail,weight,head_type,tail_type,head_cui,tail_cui
0,*NF* Bicalutamide,drug_sdoh,admission,16,drug,SDoH,C0285590,None
1,*NF* Bicalutamide,drug_sdoh,appointment,16,drug,SDoH,C0285590,None
2,*NF* Bicalutamide,drug_sdoh,aricept 10 mg,16,drug,SDoH,C0285590,None
3,*NF* Bicalutamide,drug_sdoh,assisted care facility,16,drug,SDoH,C0285590,None
4,*NF* Bicalutamide,drug_sdoh,bicalutamide,16,drug,SDoH,C0285590,None
...,...,...,...,...,...,...,...,...
923384,widowed,marital_status_drug,Vitamin D,60,marital_status,drug,None,C0043491
923385,widowed,marital_status_drug,Warfarin,41,marital_status,drug,None,C0043491
923386,widowed,marital_status_drug,Zolpidem Tartrate,12,marital_status,drug,None,C0043491
923387,widowed,marital_status_drug,traZODONE,18,marital_status,drug,None,C0043491


In [13]:
sdoh = sdoh.drop(columns=['Unnamed: 0', 'x_index', 'x_source', 'y_index', 'y_source', 'pmid'])
# drop all the rows with 'relation'='subordinate'
sdoh = sdoh[sdoh['relation'] != 'subordinate'] 
# lowercase everything in 'x_type' and 'y_type'
sdoh['x_type'] = sdoh['x_type'].str.lower()
sdoh['y_type'] = sdoh['y_type'].str.lower()
# if x_type = chemical, change it to drug
sdoh.loc[sdoh['x_type'] == 'chemical', 'x_type'] = 'drug'
sdoh.loc[sdoh['y_type'] == 'sdoh factor', 'y_type'] = 'SDoH'
sdoh

,relation,x_name,x_id,x_type,y_name,y_id,y_type,probability
0,associates,dementia,MESH:D003704,disease,diagnosis,NaN,SDoH,0.5
1,associates,dementia,MESH:D003704,disease,disease-modifying treatments,NaN,SDoH,0.5
2,associates,dementia,MESH:D003704,disease,elderly care,NaN,SDoH,0.5
3,associates,dementia,MESH:D003704,disease,guidelines,NaN,SDoH,0.5
4,associates,dementia,MESH:D003704,disease,protocols,NaN,SDoH,0.5
...,...,...,...,...,...,...,...,...
2704,associates,heart disease,MESH:D006331,disease,poor nutrition,NaN,SDoH,0.5
2705,associates,respiratory diseases,MESH:D012140,disease,poor nutrition,NaN,SDoH,1.0
2706,associates,dementia,MESH:D003704,disease,disparities,NaN,SDoH,1.0
2707,associates,Dementia,MESH:D003704,disease,representation,NaN,SDoH,0.5


In [8]:
sdoh = sdoh.drop(columns=['x_index', 'x_source', 'y_index', 'y_source', 'pmid'])
# drop all the rows with 'relation'='subordinate'
sdoh = sdoh[sdoh['relation'] != 'subordinate'] 
# lowercase everything in 'x_type' and 'y_type'
sdoh['x_type'] = sdoh['x_type'].str.lower()
sdoh['y_type'] = sdoh['y_type'].str.lower()
# if x_type = chemical, change it to drug
sdoh.loc[sdoh['x_type'] == 'chemical', 'x_type'] = 'drug'
sdoh.loc[sdoh['y_type'] == 'sdoh factor', 'y_type'] = 'SDoH'
sdoh

,relation,x_name,x_id,x_type,y_name,y_id,y_type,probability
0,associates,dementia (disease),1627,disease,diagnosis,NaN,SDoH,0.5
1,associates,dementia (disease),1627,disease,disease-modifying treatments,NaN,SDoH,0.5
2,associates,dementia (disease),1627,disease,elderly care,NaN,SDoH,0.5
3,associates,dementia (disease),1627,disease,guidelines,NaN,SDoH,0.5
4,associates,dementia (disease),1627,disease,protocols,NaN,SDoH,0.5
...,...,...,...,...,...,...,...,...
2704,associates,heart disease,5267,disease,poor nutrition,NaN,SDoH,0.5
2705,associates,respiratory system disease,5087,disease,poor nutrition,NaN,SDoH,1.0
2706,associates,dementia (disease),1627,disease,disparities,NaN,SDoH,1.0
2707,associates,dementia (disease),1627,disease,representation,NaN,SDoH,0.5


In [9]:
sdoh = sdoh.rename(columns={'x_name': 'head', 'y_name': 'tail', 'x_type': 'head_type', 'y_type': 'tail_type', 'probability':'weight'})
# for all values in 'weight', times it by 100
sdoh['weight'] = sdoh['weight'] * 100
# change 'relation' to x_type_y_type
sdoh['relation'] = sdoh['head_type'] + '_' + sdoh['tail_type']
# remove 'MESH:' from the beginning of each element in the list
sdoh['x_id'] = sdoh['x_id'].str.replace('MESH:', '')
sdoh

,relation,head,x_id,head_type,tail,y_id,tail_type,weight
0,disease_SDoH,dementia (disease),1627,disease,diagnosis,NaN,SDoH,50.0
1,disease_SDoH,dementia (disease),1627,disease,disease-modifying treatments,NaN,SDoH,50.0
2,disease_SDoH,dementia (disease),1627,disease,elderly care,NaN,SDoH,50.0
3,disease_SDoH,dementia (disease),1627,disease,guidelines,NaN,SDoH,50.0
4,disease_SDoH,dementia (disease),1627,disease,protocols,NaN,SDoH,50.0
...,...,...,...,...,...,...,...,...
2704,disease_SDoH,heart disease,5267,disease,poor nutrition,NaN,SDoH,50.0
2705,disease_SDoH,respiratory system disease,5087,disease,poor nutrition,NaN,SDoH,100.0
2706,disease_SDoH,dementia (disease),1627,disease,disparities,NaN,SDoH,100.0
2707,disease_SDoH,dementia (disease),1627,disease,representation,NaN,SDoH,50.0


In [10]:
# get unique 'head_type'
h = sdoh['head_type'].unique()
h

array(['disease', 'species', 'drug', 'gene/protein', 'variant',
       'cellline'], dtype=object)

In [15]:
group2 = sdoh
df1 = pd.read_csv('mapping/sdoh_drug.csv', low_memory=False)
df2 = pd.read_csv('mapping/sdoh_disease.csv', low_memory=False)
group2['head_id'] = None
group2['tail_id'] = None
for index, row in group2.iterrows():
    if row['head_type'] == 'drug':
        head = row['x_id']
        cui = df1[df1['name'] == head]['cui'].values
        if len(cui) > 0:
            group2.at[index, 'head_id'] = cui[0]
    elif row['head_type'] == 'disease':
        head = row['x_id']
        cui = df2[df2['name'] == head]['cui'].values
        if len(cui) > 0:
            group2.at[index, 'head_id'] = cui[0]
            
group2

,relation,head,x_id,head_type,tail,y_id,tail_type,weight,head_id,tail_id
0,disease_SDoH,dementia,D003704,disease,diagnosis,NaN,SDoH,50.0,C0338630,None
1,disease_SDoH,dementia,D003704,disease,disease-modifying treatments,NaN,SDoH,50.0,C0338630,None
2,disease_SDoH,dementia,D003704,disease,elderly care,NaN,SDoH,50.0,C0338630,None
3,disease_SDoH,dementia,D003704,disease,guidelines,NaN,SDoH,50.0,C0338630,None
4,disease_SDoH,dementia,D003704,disease,protocols,NaN,SDoH,50.0,C0338630,None
...,...,...,...,...,...,...,...,...,...,...
2704,disease_SDoH,heart disease,D006331,disease,poor nutrition,NaN,SDoH,50.0,C0018799,None
2705,disease_SDoH,respiratory diseases,D012140,disease,poor nutrition,NaN,SDoH,100.0,C0035242,None
2706,disease_SDoH,dementia,D003704,disease,disparities,NaN,SDoH,100.0,C0338630,None
2707,disease_SDoH,Dementia,D003704,disease,representation,NaN,SDoH,50.0,C0338630,None


In [ ]:
group2.loc[group2['head_type'] == 'gene', 'x_type'] = 'gene/protein'

In [19]:
group2['relation'] = group2['head_type'] + '_' + sdoh['tail_type']
group2

,relation,head,x_id,head_type,tail,y_id,tail_type,weight,head_id,tail_id,x_type
0,disease_SDoH,dementia,D003704,disease,diagnosis,NaN,SDoH,50.0,C0338630,None,NaN
1,disease_SDoH,dementia,D003704,disease,disease-modifying treatments,NaN,SDoH,50.0,C0338630,None,NaN
2,disease_SDoH,dementia,D003704,disease,elderly care,NaN,SDoH,50.0,C0338630,None,NaN
3,disease_SDoH,dementia,D003704,disease,guidelines,NaN,SDoH,50.0,C0338630,None,NaN
4,disease_SDoH,dementia,D003704,disease,protocols,NaN,SDoH,50.0,C0338630,None,NaN
...,...,...,...,...,...,...,...,...,...,...,...
2704,disease_SDoH,heart disease,D006331,disease,poor nutrition,NaN,SDoH,50.0,C0018799,None,NaN
2705,disease_SDoH,respiratory diseases,D012140,disease,poor nutrition,NaN,SDoH,100.0,C0035242,None,NaN
2706,disease_SDoH,dementia,D003704,disease,disparities,NaN,SDoH,100.0,C0338630,None,NaN
2707,disease_SDoH,Dementia,D003704,disease,representation,NaN,SDoH,50.0,C0338630,None,NaN


In [23]:
pkg = pd.read_csv(r'C:\data\RA\shenlab\SDoH&KG\data\kg.csv')
pkg = pkg.drop(columns=['display_relation', 'x_index', 'x_source', 'y_index', 'y_source'])
# lowercase everything in 'x_type' and 'y_type'
pkg['x_type'] = pkg['x_type'].str.lower()
pkg['y_type'] = pkg['y_type'].str.lower()
pkg['relation'] = pkg['x_type'] + '_' + pkg['y_type']
pkg = pkg.rename(columns={'x_name': 'head', 'y_name': 'tail', 'x_type': 'head_type', 'y_type': 'tail_type'})
pkg

C:\Users\Tianqi\AppData\Local\Temp\ipykernel_13140\935755359.py:1: DtypeWarning: Columns (3,8) have mixed types. Specify dtype option on import or set low_memory=False.
  pkg = pd.read_csv(r'C:\data\RA\shenlab\SDoH&KG\data\kg.csv')


,relation,x_id,head_type,head,y_id,tail_type,tail
0,gene/protein_gene/protein,9796,gene/protein,PHYHIP,56992,gene/protein,KIF15
1,gene/protein_gene/protein,7918,gene/protein,GPANK1,9240,gene/protein,PNMA1
2,gene/protein_gene/protein,8233,gene/protein,ZRSR2,23548,gene/protein,TTC33
3,gene/protein_gene/protein,4899,gene/protein,NRF1,11253,gene/protein,MAN1B1
4,gene/protein_gene/protein,5297,gene/protein,PI4KA,8601,gene/protein,RGS20
...,...,...,...,...,...,...,...
8100493,anatomy_gene/protein,4720,anatomy,cerebellar vermis,140,gene/protein,ADORA3
8100494,anatomy_gene/protein,1377,anatomy,quadriceps femoris,105378952,gene/protein,KLF18
8100495,anatomy_gene/protein,1379,anatomy,vastus lateralis,105378952,gene/protein,KLF18
8100496,anatomy_gene/protein,2084,anatomy,heart left ventricle,105378952,gene/protein,KLF18


In [24]:
# get unique 'head_type' and 'tail_type'
h = pkg['head_type'].unique()
t = pkg['tail_type'].unique()
h, t

(array(['gene/protein', 'drug', 'effect/phenotype', 'disease',
        'biological_process', 'molecular_function', 'cellular_component',
        'exposure', 'pathway', 'anatomy'], dtype=object),
 array(['gene/protein', 'disease', 'drug', 'effect/phenotype',
        'biological_process', 'molecular_function', 'cellular_component',
        'exposure', 'pathway', 'anatomy'], dtype=object))

In [25]:
group3 = pkg
df1 = pd.read_csv('mapping/pkg_drug.csv', low_memory=False)
df2 = pd.read_csv('mapping/pkg_disease.csv', low_memory=False)
group3['head_id'] = None
group3['tail_id'] = None
for index, row in group3.iterrows():
    if row['head_type'] == 'drug':
        head = row['x_id']
        cui = df1[df1['name'] == head]['cui'].values
        if len(cui) > 0:
            group3.at[index, 'head_id'] = cui[0]
    elif row['tail_type'] == 'drug':
        tail = row['y_id']
        cui = df1[df1['name'] == head]['cui'].values
        if len(cui) > 0:
            group3.at[index, 'tail_id'] = cui[0]
    elif row['head_type'] == 'disease':
        head = row['head']
        cui = df2[df2['name'] == head]['cui'].values
        if len(cui) > 0:
            group3.at[index, 'head_id'] = cui[0]
    elif row['tail_type'] == 'disease':
        tail = row['tail']
        cui = df2[df2['name'] == head]['cui'].values
        if len(cui) > 0:
            group3.at[index, 'tail_id'] = cui[0]
            
group3

,relation,x_id,head_type,head,y_id,tail_type,tail,head_id,tail_id
0,gene/protein_gene/protein,9796,gene/protein,PHYHIP,56992,gene/protein,KIF15,None,None
1,gene/protein_gene/protein,7918,gene/protein,GPANK1,9240,gene/protein,PNMA1,None,None
2,gene/protein_gene/protein,8233,gene/protein,ZRSR2,23548,gene/protein,TTC33,None,None
3,gene/protein_gene/protein,4899,gene/protein,NRF1,11253,gene/protein,MAN1B1,None,None
4,gene/protein_gene/protein,5297,gene/protein,PI4KA,8601,gene/protein,RGS20,None,None
...,...,...,...,...,...,...,...,...,...
8100493,anatomy_gene/protein,4720,anatomy,cerebellar vermis,140,gene/protein,ADORA3,None,None
8100494,anatomy_gene/protein,1377,anatomy,quadriceps femoris,105378952,gene/protein,KLF18,None,None
8100495,anatomy_gene/protein,1379,anatomy,vastus lateralis,105378952,gene/protein,KLF18,None,None
8100496,anatomy_gene/protein,2084,anatomy,heart left ventricle,105378952,gene/protein,KLF18,None,None


In [26]:
# save the 3 groups to 3 csv files
group1.to_csv('mimicKG1.csv', index=False)
group2.to_csv('sdoh1.csv', index=False)
group3.to_csv('pkg1.csv', index=False)

In [17]:
import pandas as pd

# Initialize the DataFrame with the specified columns
subtype_type = pd.DataFrame(columns=['relation', 'head', 'tail', 'head_type', 'tail_type'])

# Define the types and sub-categories
types = ['Healthcare access and quality', 'Education access and quality', 'Social and community context', 'Economic stability', 'Neighborhood and built environment']
sub_categories = ["Access to Health Services; Access to Primary Care; Health Literacy; Health Insurance",
                  "Early Childhood Development and Education; Enrollment in Primary Education; Enrollment in Secondary Education; Enrollment in Higher Education; Language and Literacy; Uneducated",
                  "Civic Participation; Racism and Discrimination; Incarceration; Social Cohesion; Involved in Illegal activities",
                  "Employment; Income; Wealthy; Poverty; Transportation",
                  "Access to Foods That Support Healthy Dietary Patterns; Crime and Violence; Environmental Conditions; Quality of Housing; Homelessness; Food Insecurity; Quality of Neighborhoods"]


# Use a list to accumulate all rows to concatenate at the end
rows = []

# Iterate to build each row
for i in range(len(types)):
    #print(f"Processing type: {types[i]}")  # Debugging print
    for j in sub_categories[i].split('; '):
        #print(f"Adding row: {j}")  # Debugging print
        rows.append({
            'relation': 'subordinate1', 
            'head': j.strip(), 
            'tail': types[i], 
            'head_type': 'subtype', 
            'tail_type': 'type'
        })

# Check if rows are populated correctly
#print("Total rows accumulated:", len(rows))  # Debugging print

# Concatenate the accumulated rows into the DataFrame
subtype_type = pd.concat([subtype_type, pd.DataFrame(rows)], ignore_index=True)

# Display the DataFrame
subtype_type.head()


,relation,head,tail,head_type,tail_type
0,subordinate1,Access to Health Services,Healthcare access and quality,subtype,type
1,subordinate1,Access to Primary Care,Healthcare access and quality,subtype,type
2,subordinate1,Health Literacy,Healthcare access and quality,subtype,type
3,subordinate1,Health Insurance,Healthcare access and quality,subtype,type
4,subordinate1,Early Childhood Development and Education,Education access and quality,subtype,type


In [6]:
group1 = pd.read_csv('mid\mimicKG1.csv')
group2 = pd.read_csv('mid\sdoh1.csv')
group3 = pd.read_csv('mid\pkg1.csv')

C:\Users\Tianqi\AppData\Local\Temp\ipykernel_22684\434932315.py:1: DtypeWarning: Columns (0,7) have mixed types. Specify dtype option on import or set low_memory=False.
  group1 = pd.read_csv('mid\mimicKG1.csv')
C:\Users\Tianqi\AppData\Local\Temp\ipykernel_22684\434932315.py:3: DtypeWarning: Columns (1,4,7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  group3 = pd.read_csv('mid\pkg1.csv')


In [15]:
sub_cate = [['Access to Health Services', 'Access to Primary Care', 'Health Literacy', 'Health Insurance'],
                  ['Early Childhood Development and Education', 'Enrollment in Primary Education', 'Enrollment in Secondary Education', 'Enrollment in Higher Education', 'Language and Literacy', 'Uneducated'],
                  ['Civic Participation', 'Racism and Discrimination', 'Incarceration', 'Social Cohesion', 'Involved in Illegal activities'],
                  ['Employment', 'Income', 'Wealthy', 'Poverty', 'Transportation'],
                  ['Access to Foods That Support Healthy Dietary Patterns', 'Crime and Violence', 'Environmental Conditions', 'Quality of Housing', 'Homelessness', 'Food Insecurity', 'Quality of Neighborhoods']]


In [18]:
mimic_sdoh = pd.read_csv('mid\mimic_sdoh_class.csv')

# Initialize 'head' and 'tail' columns with None
mimic_sdoh['head'] = None
mimic_sdoh['tail'] = None

# Iterate over the DataFrame rows
for index, row in mimic_sdoh.iterrows():
    # Print the values being accessed for debugging
    #print(f"Row {index}: type={row['type']}, subtype={row['subtype']}")

    # Ensure type and subtype are within range before accessing
    if row['type'] - 1 >= len(sub_cate) or row['subtype'] - 1 >= len(sub_cate[row['type'] - 1]):
        print(f"Out of range at index {index}: type={row['type']}, subtype={row['subtype']}")
        continue  # Skip this iteration if out of range

    # Assign 'head' and 'tail' values
    mimic_sdoh.at[index, 'head'] = sub_cate[row['type'] - 1][row['subtype'] - 1]
    mimic_sdoh.at[index, 'tail'] = types[row['type'] - 1]

# Display the DataFrame
mimic_sdoh


,text,type,subtype,head,tail
0,multidisciplinary rounds,2,1,Early Childhood Development and Education,Education access and quality
1,gastroenterolgoy code dnr/dni(confirmed,5,3,Environmental Conditions,Neighborhood and built environment
2,nursing station,3,5,Involved in Illegal activities,Social and community context
3,bisacodyl dulcolax,4,5,Transportation,Economic stability
4,code physical examination,5,3,Environmental Conditions,Neighborhood and built environment
...,...,...,...,...,...
5284,services,1,1,Access to Health Services,Healthcare access and quality
5285,sedimentation,1,1,Access to Health Services,Healthcare access and quality
5286,descending/abdominal aorta ct surgery,1,1,Access to Health Services,Healthcare access and quality
5287,discharge acute,1,1,Access to Health Services,Healthcare access and quality


In [19]:
sdoh_sdoh = pd.read_csv('mid\sdoh_sdoh_class.csv')

# Initialize 'head' and 'tail' columns with None
sdoh_sdoh['head'] = None
sdoh_sdoh['tail'] = None
# if 'type' and 'subtype' are not NaN, convert them to integers
sdoh_sdoh['type'] = sdoh_sdoh['type'].fillna(-1).astype(int)
sdoh_sdoh['subtype'] = sdoh_sdoh['subtype'].fillna(-1).astype(int)
# Iterate over the DataFrame rows
for index, row in sdoh_sdoh.iterrows():
    # Print the values being accessed for debugging
    #print(f"Row {index}: type={row['type']}, subtype={row['subtype']}")

    # Ensure type and subtype are within range before accessing
    if row['type'] - 1 >= len(sub_cate) or row['subtype'] - 1 >= len(sub_cate[row['type'] - 1]):
        print(f"Out of range at index {index}: type={row['type']}, subtype={row['subtype']}")
        continue  # Skip this iteration if out of range

    # Assign 'head' and 'tail' values
    sdoh_sdoh.at[index, 'head'] = sub_cate[row['type'] - 1][row['subtype'] - 1]
    sdoh_sdoh.at[index, 'tail'] = types[row['type'] - 1]

# if 'type' and 'subtype' are -1, clear the 'head' and 'tail' columns
sdoh_sdoh.loc[sdoh_sdoh['type'] == -1, ['head', 'tail']] = None

# Display the DataFrame
sdoh_sdoh


,text,type,subtype,head,tail
0,diagnosis,1,1,Access to Health Services,Healthcare access and quality
1,disease-modifying treatments,1,1,Access to Health Services,Healthcare access and quality
2,elderly care,1,1,Access to Health Services,Healthcare access and quality
3,guidelines,1,1,Access to Health Services,Healthcare access and quality
4,protocols,1,1,Access to Health Services,Healthcare access and quality
...,...,...,...,...,...
1370,ethnic/race group differences,-1,-1,None,None
1371,non-Hispanic Black,-1,-1,None,None
1372,utility hardship,4,2,Income,Economic stability
1373,income segregation,4,2,Income,Economic stability


In [51]:
# concatenate mimic_sdoh and sdoh_sdoh
sdoh_type = pd.concat([mimic_sdoh, sdoh_sdoh], ignore_index=True)
# drop duplicates
sdoh_type = sdoh_type.drop_duplicates()
# save to a csv file
sdoh_type.to_csv('sdoh_type.csv', index=False)


In [20]:
subtype_type['weight'] = np.nan
subtype_type['head_id'] = np.nan
subtype_type['tail_id'] = np.nan
subtype_type.head()

,relation,head,tail,head_type,tail_type,weight,head_id,tail_id
0,subordinate1,Access to Health Services,Healthcare access and quality,subtype,type,NaN,NaN,NaN
1,subordinate1,Access to Primary Care,Healthcare access and quality,subtype,type,NaN,NaN,NaN
2,subordinate1,Health Literacy,Healthcare access and quality,subtype,type,NaN,NaN,NaN
3,subordinate1,Health Insurance,Healthcare access and quality,subtype,type,NaN,NaN,NaN
4,subordinate1,Early Childhood Development and Education,Education access and quality,subtype,type,NaN,NaN,NaN


In [22]:
import numpy as np
sdoh_type = pd.read_csv('mid\sdoh_type.csv')
sdoh_type = sdoh_type.drop(columns=['tail', 'type'])
# rename the columns, head_id to head_cui, tail_id to tail_cui
sdoh_type = sdoh_type.rename(columns={'head': 'tail'})
sdoh_type = sdoh_type.rename(columns={'text': 'head', 'subtype': 'tail_id'})
# add columns 'relation' = 'subordinate0', 'head_type' = 'SDoH', 'tail_type' = 'subtype'
sdoh_type['relation'] = 'subordinate0'
sdoh_type['head_type'] = 'SDoH'
sdoh_type['tail_type'] = 'subtype'
sdoh_type['weight'] = np.nan
sdoh_type['head_id'] = np.nan
sdoh_type

,head,tail_id,tail,relation,head_type,tail_type,weight,head_id
0,multidisciplinary rounds,1,Early Childhood Development and Education,subordinate0,SDoH,subtype,NaN,NaN
1,gastroenterolgoy code dnr/dni(confirmed,3,Environmental Conditions,subordinate0,SDoH,subtype,NaN,NaN
2,nursing station,5,Involved in Illegal activities,subordinate0,SDoH,subtype,NaN,NaN
3,bisacodyl dulcolax,5,Transportation,subordinate0,SDoH,subtype,NaN,NaN
4,code physical examination,3,Environmental Conditions,subordinate0,SDoH,subtype,NaN,NaN
...,...,...,...,...,...,...,...,...
6570,ethnic/race group differences,2,Racism and Discrimination,subordinate0,SDoH,subtype,NaN,NaN
6571,non-Hispanic Black,-1,Ethnicity,subordinate0,SDoH,subtype,NaN,NaN
6572,utility hardship,2,Income,subordinate0,SDoH,subtype,NaN,NaN
6573,income segregation,2,Income,subordinate0,SDoH,subtype,NaN,NaN


In [74]:
# concatenate subtype_type and sdoh_type
typedf = pd.concat([subtype_type, sdoh_type], ignore_index=True)
# drop duplicates
typedf = typedf.drop_duplicates()
# save to a csv file
typedf.to_csv('typedf.csv', index=False)
typedf

,relation,head,tail,head_type,tail_type,weight,head_id,tail_id
0,subordinate1,Access to Health Services,Healthcare access and quality,subtype,type,NaN,NaN,NaN
1,subordinate1,Access to Primary Care,Healthcare access and quality,subtype,type,NaN,NaN,NaN
2,subordinate1,Health Literacy,Healthcare access and quality,subtype,type,NaN,NaN,NaN
3,subordinate1,Health Insurance,Healthcare access and quality,subtype,type,NaN,NaN,NaN
4,subordinate1,Early Childhood Development and Education,Education access and quality,subtype,type,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
6597,subordinate0,ethnic/race group differences,Racism and Discrimination,SDoH,subtype,NaN,NaN,2.0
6598,subordinate0,non-Hispanic Black,Ethnicity,SDoH,subtype,NaN,NaN,-1.0
6599,subordinate0,utility hardship,Income,SDoH,subtype,NaN,NaN,2.0
6600,subordinate0,income segregation,Income,SDoH,subtype,NaN,NaN,2.0


In [7]:
group1

,head,relation,tail,weight,head_type,tail_type,head_cui,tail_cui
0,*NF* Bicalutamide,drug_sdoh,admission,16,drug,SDoH,C0285590,NaN
1,*NF* Bicalutamide,drug_sdoh,appointment,16,drug,SDoH,C0285590,NaN
2,*NF* Bicalutamide,drug_sdoh,aricept 10 mg,16,drug,SDoH,C0285590,NaN
3,*NF* Bicalutamide,drug_sdoh,assisted care facility,16,drug,SDoH,C0285590,NaN
4,*NF* Bicalutamide,drug_sdoh,bicalutamide,16,drug,SDoH,C0285590,NaN
...,...,...,...,...,...,...,...,...
923384,widowed,marital_status_drug,Vitamin D,60,marital_status,drug,NaN,C0043491
923385,widowed,marital_status_drug,Warfarin,41,marital_status,drug,NaN,C0043491
923386,widowed,marital_status_drug,Zolpidem Tartrate,12,marital_status,drug,NaN,C0043491
923387,widowed,marital_status_drug,traZODONE,18,marital_status,drug,NaN,C0043491


In [23]:
# get unique 'relation'
r1 = group1['relation'].unique()
r2 = group2['relation'].unique()

r1, r2

(array(['drug_sdoh', 'drug_disease', 'disease_sdoh', 'age_disease',
        'age_drug', 'ethnicity_drug', 'ethnicity_disease', 'religion_drug',
        'religion_disease', 'marital_status_disease',
        'marital_status_drug', 'gender_drug', 'gender_disease',
        'insurance_drug', 'insurance_disease'], dtype=object),
 array(['disease_SDoH', 'species_SDoH', 'drug_SDoH', 'gene_SDoH',
        'variant_SDoH', 'cellline_SDoH'], dtype=object))

In [8]:
# reorder the columns
group2 = group2[['head', 'relation', 'tail', 'weight', 'head_type', 'tail_type', 'head_id', 'tail_id', 'x_id']]
# lowercase everything in 'relation'
group2['relation'] = group2['relation'].str.lower()
group2 = group2.rename(columns={'head_id': 'head_cui', 'tail_id': 'tail_cui'})
group2

,head,relation,tail,weight,head_type,tail_type,head_cui,tail_cui,x_id
0,dementia,disease_sdoh,diagnosis,50.0,disease,SDoH,C0338630,NaN,D003704
1,dementia,disease_sdoh,disease-modifying treatments,50.0,disease,SDoH,C0338630,NaN,D003704
2,dementia,disease_sdoh,elderly care,50.0,disease,SDoH,C0338630,NaN,D003704
3,dementia,disease_sdoh,guidelines,50.0,disease,SDoH,C0338630,NaN,D003704
4,dementia,disease_sdoh,protocols,50.0,disease,SDoH,C0338630,NaN,D003704
...,...,...,...,...,...,...,...,...,...
2703,heart disease,disease_sdoh,poor nutrition,50.0,disease,SDoH,C0018799,NaN,D006331
2704,respiratory diseases,disease_sdoh,poor nutrition,100.0,disease,SDoH,C0035242,NaN,D012140
2705,dementia,disease_sdoh,disparities,100.0,disease,SDoH,C0338630,NaN,D003704
2706,Dementia,disease_sdoh,representation,50.0,disease,SDoH,C0338630,NaN,D003704


In [12]:
# concatenate group1 and group2
group = pd.concat([group1, group2], ignore_index=True)
# drop duplicates if relation, head_cui, tail are the same
#group = group.drop_duplicates(subset=['relation', 'head_cui', 'tail'])

# Step 1: Identify rows where either 'head_cui' or 'tail_cui' is NaN
mask_na_cui = group['head_cui'].isna() | group['tail_cui'].isna()

# Step 2: Deduplicate rows with NaN in 'head_cui' or 'tail_cui' by 'head' and 'tail'
dedup_na_cui = group[mask_na_cui].drop_duplicates(subset=['head', 'tail'], keep='first')

# Step 3: Deduplicate rows where both 'head_cui' and 'tail_cui' are present by 'head_cui' and 'tail_cui'
dedup_with_cui = group[~mask_na_cui].drop_duplicates(subset=['head_cui', 'tail_cui'], keep='first')

# Step 4: Concatenate the deduplicated DataFrames
biggroup_deduplicated = pd.concat([dedup_na_cui, dedup_with_cui], ignore_index=True)

# Step 5: Reset index (optional) to keep things clean
group = biggroup_deduplicated.reset_index(drop=True)

group = group.drop_duplicates()
#reset the index
group = group.reset_index(drop=True)
group

,head,relation,tail,weight,head_type,tail_type,head_cui,tail_cui,x_id
0,*NF* Bicalutamide,drug_sdoh,admission,16.0,drug,SDoH,C0285590,NaN,NaN
1,*NF* Bicalutamide,drug_sdoh,appointment,16.0,drug,SDoH,C0285590,NaN,NaN
2,*NF* Bicalutamide,drug_sdoh,aricept 10 mg,16.0,drug,SDoH,C0285590,NaN,NaN
3,*NF* Bicalutamide,drug_sdoh,assisted care facility,16.0,drug,SDoH,C0285590,NaN,NaN
4,*NF* Bicalutamide,drug_sdoh,bicalutamide,16.0,drug,SDoH,C0285590,NaN,NaN
...,...,...,...,...,...,...,...,...,...
925964,heart disease,disease_sdoh,poor nutrition,50.0,disease,SDoH,C0018799,NaN,D006331
925965,respiratory diseases,disease_sdoh,poor nutrition,100.0,disease,SDoH,C0035242,NaN,D012140
925966,dementia,disease_sdoh,disparities,100.0,disease,SDoH,C0338630,NaN,D003704
925967,Dementia,disease_sdoh,representation,50.0,disease,SDoH,C0338630,NaN,D003704


In [13]:
# save to a csv file
group.to_csv('sdohkg(2in1).csv', index=False)

In [83]:
r = group['relation'].unique()
r

array(['drug_sdoh', 'drug_disease', 'disease_sdoh', 'age_disease',
       'age_drug', 'ethnicity_drug', 'ethnicity_disease', 'religion_drug',
       'religion_disease', 'marital_status_disease',
       'marital_status_drug', 'gender_drug', 'gender_disease',
       'insurance_drug', 'insurance_disease', 'species_sdoh', 'gene_sdoh',
       'variant_sdoh', 'cellline_sdoh'], dtype=object)

In [92]:
r3 = group3['relation'].unique()
r3

array(['gene_gene', 'drug_gene', 'drug_disease', 'drug_drug',
       'gene_effect/phenotype', 'effect/phenotype_effect/phenotype',
       'disease_effect/phenotype', 'effect/phenotype_disease',
       'gene_disease', 'disease_disease', 'drug_effect/phenotype',
       'biological_process_biological_process',
       'molecular_function_molecular_function',
       'cellular_component_cellular_component', 'gene_molecular_function',
       'gene_cellular_component', 'gene_biological_process',
       'exposure_gene', 'exposure_disease', 'exposure_exposure',
       'exposure_biological_process', 'exposure_molecular_function',
       'exposure_cellular_component', 'pathway_pathway', 'gene_pathway',
       'anatomy_anatomy', 'gene_anatomy', 'gene_drug', 'disease_drug',
       'effect/phenotype_gene', 'disease_gene', 'effect/phenotype_drug',
       'molecular_function_gene', 'cellular_component_gene',
       'biological_process_gene', 'gene_exposure', 'disease_exposure',
       'biological_proce

In [93]:
# get the same relation in r and r3
r4 = [x for x in r if x in r3]
r4

['drug_disease']

In [27]:
group

,head,relation,tail,weight,head_type,tail_type,head_cui,tail_cui,x_id
0,*NF* Bicalutamide,drug_sdoh,admission,16.0,drug,SDoH,C0285590,NaN,NaN
1,*NF* Bicalutamide,drug_sdoh,appointment,16.0,drug,SDoH,C0285590,NaN,NaN
2,*NF* Bicalutamide,drug_sdoh,aricept 10 mg,16.0,drug,SDoH,C0285590,NaN,NaN
3,*NF* Bicalutamide,drug_sdoh,assisted care facility,16.0,drug,SDoH,C0285590,NaN,NaN
4,*NF* Bicalutamide,drug_sdoh,bicalutamide,16.0,drug,SDoH,C0285590,NaN,NaN
...,...,...,...,...,...,...,...,...,...
810420,heart disease,disease_sdoh,poor nutrition,50.0,disease,SDoH,C0018799,NaN,D006331
810421,respiratory diseases,disease_sdoh,poor nutrition,100.0,disease,SDoH,C0035242,NaN,D012140
810422,dementia,disease_sdoh,disparities,100.0,disease,SDoH,C0338630,NaN,D003704
810423,Dementia,disease_sdoh,representation,50.0,disease,SDoH,C0338630,NaN,D003704


In [28]:
group3['weight'] = np.nan
group3 = group3[['head', 'relation', 'tail', 'weight', 'head_type', 'tail_type', 'head_id', 'tail_id', 'x_id','y_id']]
group3 = group3.rename(columns={'head_id': 'head_cui', 'tail_id': 'tail_cui'})
# change all the gene/protein to gene
group3.loc[group3['head_type'] == 'gene/protein', 'head_type'] = 'gene'
group3.loc[group3['tail_type'] == 'gene/protein', 'tail_type'] = 'gene'
# change relation to head_type_tail_type
group3['relation'] = group3['head_type'] + '_' + group3['tail_type']
group3

,head,relation,tail,weight,head_type,tail_type,head_cui,tail_cui,x_id,y_id
0,PHYHIP,gene_gene,KIF15,NaN,gene,gene,NaN,NaN,9796,56992
1,GPANK1,gene_gene,PNMA1,NaN,gene,gene,NaN,NaN,7918,9240
2,ZRSR2,gene_gene,TTC33,NaN,gene,gene,NaN,NaN,8233,23548
3,NRF1,gene_gene,MAN1B1,NaN,gene,gene,NaN,NaN,4899,11253
4,PI4KA,gene_gene,RGS20,NaN,gene,gene,NaN,NaN,5297,8601
...,...,...,...,...,...,...,...,...,...,...
8100493,cerebellar vermis,anatomy_gene,ADORA3,NaN,anatomy,gene,NaN,NaN,4720,140
8100494,quadriceps femoris,anatomy_gene,KLF18,NaN,anatomy,gene,NaN,NaN,1377,105378952
8100495,vastus lateralis,anatomy_gene,KLF18,NaN,anatomy,gene,NaN,NaN,1379,105378952
8100496,heart left ventricle,anatomy_gene,KLF18,NaN,anatomy,gene,NaN,NaN,2084,105378952


In [98]:
rd = ['gene_gene', 'drug_gene', 'drug_disease', 'drug_drug',
       'gene_effect/phenotype', 'effect/phenotype_effect/phenotype',
       'disease_effect/phenotype',
       'gene_disease', 'disease_disease', 'drug_effect/phenotype',
       'biological_process_biological_process',
       'molecular_function_molecular_function',
       'cellular_component_cellular_component', 'gene_molecular_function',
       'gene_cellular_component', 'gene_biological_process',
       'exposure_gene', 'exposure_disease', 'exposure_exposure',
       'exposure_biological_process', 'exposure_molecular_function',
       'exposure_cellular_component', 'pathway_pathway', 'gene_pathway',
       'anatomy_anatomy', 'gene_anatomy']

In [100]:
group4 = group3.drop_duplicates()
# if relation is in rd, keep it
group4 = group4[group4['relation'].isin(rd)]
group4 = group4.reset_index(drop=True)
group4

,head,relation,tail,weight,head_type,tail_type,head_cui,tail_cui,x_id,y_id
0,PHYHIP,gene_gene,KIF15,NaN,gene,gene,NaN,NaN,9796,56992
1,GPANK1,gene_gene,PNMA1,NaN,gene,gene,NaN,NaN,7918,9240
2,ZRSR2,gene_gene,TTC33,NaN,gene,gene,NaN,NaN,8233,23548
3,NRF1,gene_gene,MAN1B1,NaN,gene,gene,NaN,NaN,4899,11253
4,PI4KA,gene_gene,RGS20,NaN,gene,gene,NaN,NaN,5297,8601
...,...,...,...,...,...,...,...,...,...,...
5848019,digestive tract junction,anatomy_anatomy,intestinal junction,NaN,anatomy,anatomy,NaN,NaN,1100000,8410024
5848020,zone of organ,anatomy_anatomy,transition zone of prostate,NaN,anatomy,anatomy,NaN,NaN,34944,8410025
5848021,zone of organ,anatomy_anatomy,peripheral zone of prostate,NaN,anatomy,anatomy,NaN,NaN,34944,8410026
5848022,zone of organ,anatomy_anatomy,central zone of prostate,NaN,anatomy,anatomy,NaN,NaN,34944,8410027


In [27]:
# Step 1: Identify rows where either 'head_cui' or 'tail_cui' is NaN
mask_na_cui = group4['head_cui'].isna() | group4['tail_cui'].isna()

# Step 2: Deduplicate rows with NaN in 'head_cui' or 'tail_cui' by 'head' and 'tail'
dedup_na_cui = group4[mask_na_cui].drop_duplicates(subset=['head', 'tail'], keep='first')

# Step 3: Deduplicate rows where both 'head_cui' and 'tail_cui' are present by 'head_cui' and 'tail_cui'
dedup_with_cui = group4[~mask_na_cui].drop_duplicates(subset=['head_cui', 'tail_cui'], keep='first')

# Step 4: Concatenate the deduplicated DataFrames
group4_deduplicated = pd.concat([dedup_na_cui, dedup_with_cui], ignore_index=True)

# Step 5: Reset index (optional) to keep things clean
group4 = group4_deduplicated.reset_index(drop=True)
group4

,head,relation,tail,weight,head_type,tail_type,head_cui,tail_cui,x_id,y_id
0,PHYHIP,gene_gene,KIF15,NaN,gene,gene,NaN,NaN,9796,56992
1,GPANK1,gene_gene,PNMA1,NaN,gene,gene,NaN,NaN,7918,9240
2,ZRSR2,gene_gene,TTC33,NaN,gene,gene,NaN,NaN,8233,23548
3,NRF1,gene_gene,MAN1B1,NaN,gene,gene,NaN,NaN,4899,11253
4,PI4KA,gene_gene,RGS20,NaN,gene,gene,NaN,NaN,5297,8601
...,...,...,...,...,...,...,...,...,...,...
5847647,digestive tract junction,anatomy_anatomy,intestinal junction,NaN,anatomy,anatomy,NaN,NaN,1100000,8410024
5847648,zone of organ,anatomy_anatomy,transition zone of prostate,NaN,anatomy,anatomy,NaN,NaN,34944,8410025
5847649,zone of organ,anatomy_anatomy,peripheral zone of prostate,NaN,anatomy,anatomy,NaN,NaN,34944,8410026
5847650,zone of organ,anatomy_anatomy,central zone of prostate,NaN,anatomy,anatomy,NaN,NaN,34944,8410027


In [28]:
# save to a csv file
group4.to_csv('pkg(remove_back_edge).csv', index=False)


In [14]:
import pandas as pd
group4 = pd.read_csv('pkg(remove_back_edge).csv')
group = pd.read_csv('sdohkg(2in1).csv')

C:\Users\Tianqi\AppData\Local\Temp\ipykernel_22684\4193386311.py:2: DtypeWarning: Columns (6,7,8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  group4 = pd.read_csv('pkg(remove_back_edge).csv')
C:\Users\Tianqi\AppData\Local\Temp\ipykernel_22684\4193386311.py:3: DtypeWarning: Columns (0,7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  group = pd.read_csv('sdohkg(2in1).csv')


In [10]:
r4 = group4['relation'].unique()
r = group['relation'].unique()
r4, r

(array(['gene_gene', 'drug_gene', 'drug_disease', 'drug_drug',
        'gene_effect/phenotype', 'effect/phenotype_effect/phenotype',
        'disease_effect/phenotype', 'gene_disease', 'disease_disease',
        'drug_effect/phenotype', 'biological_process_biological_process',
        'molecular_function_molecular_function',
        'cellular_component_cellular_component', 'gene_molecular_function',
        'gene_cellular_component', 'gene_biological_process',
        'exposure_gene', 'exposure_disease', 'exposure_exposure',
        'exposure_biological_process', 'exposure_molecular_function',
        'exposure_cellular_component', 'pathway_pathway', 'gene_pathway',
        'anatomy_anatomy', 'gene_anatomy'], dtype=object),
 array(['drug_sdoh', 'drug_disease', 'disease_sdoh', 'age_disease',
        'age_drug', 'ethnicity_drug', 'ethnicity_disease', 'religion_drug',
        'religion_disease', 'marital_status_disease',
        'marital_status_drug', 'gender_drug', 'gender_disease',
    

In [15]:
biggroup = pd.concat([group4, group], ignore_index=True)
biggroup = biggroup.drop_duplicates()
# for relation = 'drug_disease', deduplicate by 'head_cui' and 'tail_cui', if 'head_cui' and 'tail_cui' are NaN, deduplicate by 'head' and 'tail'
# Step 1: Identify rows where either 'head_cui' or 'tail_cui' is NaN
mask_na_cui = biggroup['head_cui'].isna() | biggroup['tail_cui'].isna()

# Step 2: Deduplicate rows with NaN in 'head_cui' or 'tail_cui' by 'head' and 'tail'
dedup_na_cui = biggroup[mask_na_cui].drop_duplicates(subset=['head', 'tail'], keep='first')

# Step 3: Deduplicate rows where both 'head_cui' and 'tail_cui' are present by 'head_cui' and 'tail_cui'
dedup_with_cui = biggroup[~mask_na_cui].drop_duplicates(subset=['head_cui', 'tail_cui'], keep='first')

# Step 4: Concatenate the deduplicated DataFrames
biggroup_deduplicated = pd.concat([dedup_na_cui, dedup_with_cui], ignore_index=True)

# Step 5: Reset index (optional) to keep things clean
biggroup = biggroup_deduplicated.reset_index(drop=True)

biggroup

,head,relation,tail,weight,head_type,tail_type,head_cui,tail_cui,x_id,y_id
0,PHYHIP,gene_gene,KIF15,NaN,gene,gene,NaN,NaN,9796,56992
1,GPANK1,gene_gene,PNMA1,NaN,gene,gene,NaN,NaN,7918,9240
2,ZRSR2,gene_gene,TTC33,NaN,gene,gene,NaN,NaN,8233,23548
3,NRF1,gene_gene,MAN1B1,NaN,gene,gene,NaN,NaN,4899,11253
4,PI4KA,gene_gene,RGS20,NaN,gene,gene,NaN,NaN,5297,8601
...,...,...,...,...,...,...,...,...,...,...
6773547,heart disease,disease_sdoh,poor nutrition,50.0,disease,SDoH,C0018799,NaN,D006331,NaN
6773548,respiratory diseases,disease_sdoh,poor nutrition,100.0,disease,SDoH,C0035242,NaN,D012140,NaN
6773549,dementia,disease_sdoh,disparities,100.0,disease,SDoH,C0338630,NaN,D003704,NaN
6773550,Dementia,disease_sdoh,representation,50.0,disease,SDoH,C0338630,NaN,D003704,NaN


In [31]:
biggroup[biggroup['tail'] == 'obsolete pathologic fracture']

,head,relation,tail,weight,head_type,tail_type,head_cui,tail_cui,x_id,y_id
359318,Methylprednisolone,drug_disease,obsolete pathologic fracture,NaN,drug,disease,No results found.,NaN,DB00959,43606
359319,Desonide,drug_disease,obsolete pathologic fracture,NaN,drug,disease,C5139934,NaN,DB01260,43606
359320,Paroxetine,drug_disease,obsolete pathologic fracture,NaN,drug,disease,No results found.,NaN,DB00715,43606
359321,Prednisone,drug_disease,obsolete pathologic fracture,NaN,drug,disease,C0299792,NaN,DB00635,43606
359322,Budesonide,drug_disease,obsolete pathologic fracture,NaN,drug,disease,C4282903,NaN,DB01222,43606
359323,Rilpivirine,drug_disease,obsolete pathologic fracture,NaN,drug,disease,C0057831,NaN,DB08864,43606
359324,Prednisolone,drug_disease,obsolete pathologic fracture,NaN,drug,disease,No results found.,NaN,DB00860,43606
359325,Dexamethasone,drug_disease,obsolete pathologic fracture,NaN,drug,disease,C0085251,NaN,DB01234,43606
359326,Betamethasone,drug_disease,obsolete pathologic fracture,NaN,drug,disease,C0009316,NaN,DB00443,43606
359327,Tenofovir disoproxil,drug_disease,obsolete pathologic fracture,NaN,drug,disease,No results found.,NaN,DB00300,43606


In [16]:
# concat typedf and biggroup
typedf = pd.read_csv('typedf.csv')
kg = pd.concat([typedf, biggroup], ignore_index=True)
# drop duplicates
kg = kg.drop_duplicates()
kg = kg.reset_index(drop=True)
kg

,relation,head,tail,head_type,tail_type,weight,head_id,tail_id,head_cui,tail_cui,x_id,y_id
0,subordinate1,Access to Health Services,Healthcare access and quality,subtype,type,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,subordinate1,Access to Primary Care,Healthcare access and quality,subtype,type,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,subordinate1,Health Literacy,Healthcare access and quality,subtype,type,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,subordinate1,Health Insurance,Healthcare access and quality,subtype,type,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,subordinate1,Early Childhood Development and Education,Education access and quality,subtype,type,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
6780149,disease_sdoh,heart disease,poor nutrition,disease,SDoH,50.0,NaN,NaN,C0018799,NaN,D006331,NaN
6780150,disease_sdoh,respiratory diseases,poor nutrition,disease,SDoH,100.0,NaN,NaN,C0035242,NaN,D012140,NaN
6780151,disease_sdoh,dementia,disparities,disease,SDoH,100.0,NaN,NaN,C0338630,NaN,D003704,NaN
6780152,disease_sdoh,Dementia,representation,disease,SDoH,50.0,NaN,NaN,C0338630,NaN,D003704,NaN


In [14]:
kg[kg['tail'] == 'obsolete pathologic fracture']

,relation,head,tail,head_type,tail_type,weight,head_id,tail_id,head_cui,tail_cui,x_id,y_id
1176270,drug_disease,Methylprednisolone,obsolete pathologic fracture,drug,disease,NaN,NaN,NaN,No results found.,NaN,DB00959,43606
1176271,drug_disease,Desonide,obsolete pathologic fracture,drug,disease,NaN,NaN,NaN,C5139934,NaN,DB01260,43606
1176272,drug_disease,Paroxetine,obsolete pathologic fracture,drug,disease,NaN,NaN,NaN,No results found.,NaN,DB00715,43606
1176273,drug_disease,Prednisone,obsolete pathologic fracture,drug,disease,NaN,NaN,NaN,C0299792,NaN,DB00635,43606
1176274,drug_disease,Budesonide,obsolete pathologic fracture,drug,disease,NaN,NaN,NaN,C4282903,NaN,DB01222,43606
1176275,drug_disease,Rilpivirine,obsolete pathologic fracture,drug,disease,NaN,NaN,NaN,C0057831,NaN,DB08864,43606
1176276,drug_disease,Prednisolone,obsolete pathologic fracture,drug,disease,NaN,NaN,NaN,No results found.,NaN,DB00860,43606
1176277,drug_disease,Dexamethasone,obsolete pathologic fracture,drug,disease,NaN,NaN,NaN,C0085251,NaN,DB01234,43606
1176278,drug_disease,Betamethasone,obsolete pathologic fracture,drug,disease,NaN,NaN,NaN,C0009316,NaN,DB00443,43606
1176279,drug_disease,Tenofovir disoproxil,obsolete pathologic fracture,drug,disease,NaN,NaN,NaN,No results found.,NaN,DB00300,43606


In [17]:
# get tail_id = -1
kg1 = kg[kg['tail_id'] == -1]
# put head to list
head = kg1['head'].unique().tolist()
len(head)

53

In [18]:
kg1

,relation,head,tail,head_type,tail_type,weight,head_id,tail_id,head_cui,tail_cui,x_id,y_id
596,subordinate0,black,Ethnicity,SDoH,subtype,NaN,NaN,-1.0,NaN,NaN,NaN,NaN
1094,subordinate0,non-english,Ethnicity,SDoH,subtype,NaN,NaN,-1.0,NaN,NaN,NaN,NaN
1162,subordinate0,russian-speaking female,Ethnicity,SDoH,subtype,NaN,NaN,-1.0,NaN,NaN,NaN,NaN
1164,subordinate0,african-american,Ethnicity,SDoH,subtype,NaN,NaN,-1.0,NaN,NaN,NaN,NaN
1173,subordinate0,chinese-speaking,Ethnicity,SDoH,subtype,NaN,NaN,-1.0,NaN,NaN,NaN,NaN
1177,subordinate0,year-old chinese speaking,Ethnicity,SDoH,subtype,NaN,NaN,-1.0,NaN,NaN,NaN,NaN
1178,subordinate0,elderly white man,Ethnicity,SDoH,subtype,NaN,NaN,-1.0,NaN,NaN,NaN,NaN
2137,subordinate0,italian,Ethnicity,SDoH,subtype,NaN,NaN,-1.0,NaN,NaN,NaN,NaN
3563,subordinate0,girl,Gender,SDoH,subtype,NaN,NaN,-1.0,NaN,NaN,NaN,NaN
4127,subordinate0,jehovah,Religion,SDoH,subtype,NaN,NaN,-1.0,NaN,NaN,NaN,NaN


In [19]:
# get the rows with 'tail' in head
kg2 = kg[kg['tail'].isin(head)]
# kg2 with relation != 'subordinate0'
kg2 = kg2[kg2['relation'] != 'subordinate0']
# change tail_type to the corresponding tail in kg1
kg2['tail_type'] = kg2['tail'].apply(lambda x: kg1[kg1['head'] == x]['head_type'].values[0])
kg2

,relation,head,tail,head_type,tail_type,weight,head_id,tail_id,head_cui,tail_cui,x_id,y_id
5854561,drug_sdoh,0.45 % Sodium Chloride,elderly white man,drug,SDoH,40.0,NaN,NaN,C1725397,NaN,NaN,NaN
5854641,drug_sdoh,0.45% Sodium Chloride,african-american,drug,SDoH,53.0,NaN,NaN,C1725397,NaN,NaN,NaN
5855233,drug_sdoh,0.45% Sodium Chloride,women,drug,SDoH,20.0,NaN,NaN,C1725397,NaN,NaN,NaN
5855237,drug_sdoh,0.45% Sodium Chloride,year-old chinese speaking,drug,SDoH,29.0,NaN,NaN,C1725397,NaN,NaN,NaN
5855576,drug_sdoh,0.9% Sodium Chloride,african-american,drug,SDoH,122.0,NaN,NaN,C0982875,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
6780135,disease_sdoh,premature aging,African American,disease,SDoH,50.0,NaN,NaN,C0231341,NaN,D019588,NaN
6780138,disease_sdoh,hepatitis C co-infection,Black,disease,SDoH,100.0,NaN,NaN,C0275524,NaN,D060085,NaN
6780140,disease_sdoh,hepatitis C,Black,disease,SDoH,50.0,NaN,NaN,C0524910,NaN,D019698,NaN
6780144,disease_sdoh,knee pain,non-Hispanic Black,disease,SDoH,50.0,NaN,NaN,C0877149,NaN,D046788,NaN


In [20]:
# Step 1: Merge kg2 with kg1 on the `tail` of kg2 and `head` of kg1
merged_df = pd.merge(
    kg2, kg1[['head', 'tail']], 
    left_on='tail', right_on='head', 
    how='left'
)

# Step 2: Replace `tail_type` in kg2 with the corresponding `tail` from kg1
merged_df['tail_type'] = merged_df['tail_y'].fillna(merged_df['tail_type'])

# Step 3: Drop the extra columns from the merge (if not needed)
kg2_updated = merged_df.drop(columns=['head_y', 'tail_y'])
# rename the columns
kg2_updated = kg2_updated.rename(columns={'head_x': 'head', 'tail_x': 'tail'})
kg2_updated

,relation,head,tail,head_type,tail_type,weight,head_id,tail_id,head_cui,tail_cui,x_id,y_id
0,drug_sdoh,0.45 % Sodium Chloride,elderly white man,drug,Ethnicity,40.0,NaN,NaN,C1725397,NaN,NaN,NaN
1,drug_sdoh,0.45% Sodium Chloride,african-american,drug,Ethnicity,53.0,NaN,NaN,C1725397,NaN,NaN,NaN
2,drug_sdoh,0.45% Sodium Chloride,women,drug,Gender,20.0,NaN,NaN,C1725397,NaN,NaN,NaN
3,drug_sdoh,0.45% Sodium Chloride,year-old chinese speaking,drug,Ethnicity,29.0,NaN,NaN,C1725397,NaN,NaN,NaN
4,drug_sdoh,0.9% Sodium Chloride,african-american,drug,Ethnicity,122.0,NaN,NaN,C0982875,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
1720,disease_sdoh,premature aging,African American,disease,Ethnicity,50.0,NaN,NaN,C0231341,NaN,D019588,NaN
1721,disease_sdoh,hepatitis C co-infection,Black,disease,Ethnicity,100.0,NaN,NaN,C0275524,NaN,D060085,NaN
1722,disease_sdoh,hepatitis C,Black,disease,Ethnicity,50.0,NaN,NaN,C0524910,NaN,D019698,NaN
1723,disease_sdoh,knee pain,non-Hispanic Black,disease,Ethnicity,50.0,NaN,NaN,C0877149,NaN,D046788,NaN


In [21]:
# lower case everything in 'tail_type'
kg2_updated['tail_type'] = kg2_updated['tail_type'].str.lower()
# change 'marital status' to 'marital_status'
kg2_updated['tail_type'] = kg2_updated['tail_type'].str.replace(' ', '_')
kg2_updated[['head', 'tail']] = kg2_updated[['tail', 'head']]
kg2_updated[['head_type', 'tail_type']] = kg2_updated[['tail_type', 'head_type']]
kg2_updated[['head_id', 'tail_id']] = kg2_updated[['tail_id', 'head_id']]
kg2_updated[['head_cui', 'tail_cui']] = kg2_updated[['tail_cui', 'head_cui']]
kg2_updated[['x_id', 'y_id']] = kg2_updated[['y_id', 'x_id']]
# change 'relation' to 'head_type_tail_type'
kg2_updated['relation'] = kg2_updated['head_type'] + '_' + kg2_updated['tail_type']
kg2_updated

,relation,head,tail,head_type,tail_type,weight,head_id,tail_id,head_cui,tail_cui,x_id,y_id
0,ethnicity_drug,elderly white man,0.45 % Sodium Chloride,ethnicity,drug,40.0,NaN,NaN,NaN,C1725397,NaN,NaN
1,ethnicity_drug,african-american,0.45% Sodium Chloride,ethnicity,drug,53.0,NaN,NaN,NaN,C1725397,NaN,NaN
2,gender_drug,women,0.45% Sodium Chloride,gender,drug,20.0,NaN,NaN,NaN,C1725397,NaN,NaN
3,ethnicity_drug,year-old chinese speaking,0.45% Sodium Chloride,ethnicity,drug,29.0,NaN,NaN,NaN,C1725397,NaN,NaN
4,ethnicity_drug,african-american,0.9% Sodium Chloride,ethnicity,drug,122.0,NaN,NaN,NaN,C0982875,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
1720,ethnicity_disease,African American,premature aging,ethnicity,disease,50.0,NaN,NaN,NaN,C0231341,NaN,D019588
1721,ethnicity_disease,Black,hepatitis C co-infection,ethnicity,disease,100.0,NaN,NaN,NaN,C0275524,NaN,D060085
1722,ethnicity_disease,Black,hepatitis C,ethnicity,disease,50.0,NaN,NaN,NaN,C0524910,NaN,D019698
1723,ethnicity_disease,non-Hispanic Black,knee pain,ethnicity,disease,50.0,NaN,NaN,NaN,C0877149,NaN,D046788


In [22]:
# get unique 'relation'
r = kg2_updated['relation'].unique()
r

array(['ethnicity_drug', 'gender_drug', 'marital_status_drug',
       'religion_drug', 'ethnicity_disease', 'gender_disease',
       'marital_status_disease', 'religion_disease', 'ethnicity_species',
       'gender_species', 'ethnicity_gene', 'gender_gene'], dtype=object)

In [23]:
# remove kg1 and kg2 from kg, add kg2_updated
kg_cleaned = kg[~kg.index.isin(kg1.index) & ~kg.index.isin(kg2.index)]

# Step 2: Append the updated kg2 rows to the cleaned kg
kg_final = pd.concat([kg_cleaned, kg2_updated], ignore_index=True)
kg_final

,relation,head,tail,head_type,tail_type,weight,head_id,tail_id,head_cui,tail_cui,x_id,y_id
0,subordinate1,Access to Health Services,Healthcare access and quality,subtype,type,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,subordinate1,Access to Primary Care,Healthcare access and quality,subtype,type,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,subordinate1,Health Literacy,Healthcare access and quality,subtype,type,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,subordinate1,Health Insurance,Healthcare access and quality,subtype,type,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,subordinate1,Early Childhood Development and Education,Education access and quality,subtype,type,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
6780096,ethnicity_disease,African American,premature aging,ethnicity,disease,50.0,NaN,NaN,NaN,C0231341,NaN,D019588
6780097,ethnicity_disease,Black,hepatitis C co-infection,ethnicity,disease,100.0,NaN,NaN,NaN,C0275524,NaN,D060085
6780098,ethnicity_disease,Black,hepatitis C,ethnicity,disease,50.0,NaN,NaN,NaN,C0524910,NaN,D019698
6780099,ethnicity_disease,non-Hispanic Black,knee pain,ethnicity,disease,50.0,NaN,NaN,NaN,C0877149,NaN,D046788


In [24]:
kg_final1 = kg_final.drop_duplicates(subset=['head', 'tail'])
kg_final1



,relation,head,tail,head_type,tail_type,weight,head_id,tail_id,head_cui,tail_cui,x_id,y_id
0,subordinate1,Access to Health Services,Healthcare access and quality,subtype,type,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,subordinate1,Access to Primary Care,Healthcare access and quality,subtype,type,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,subordinate1,Health Literacy,Healthcare access and quality,subtype,type,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,subordinate1,Health Insurance,Healthcare access and quality,subtype,type,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,subordinate1,Early Childhood Development and Education,Education access and quality,subtype,type,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
6780096,ethnicity_disease,African American,premature aging,ethnicity,disease,50.0,NaN,NaN,NaN,C0231341,NaN,D019588
6780097,ethnicity_disease,Black,hepatitis C co-infection,ethnicity,disease,100.0,NaN,NaN,NaN,C0275524,NaN,D060085
6780098,ethnicity_disease,Black,hepatitis C,ethnicity,disease,50.0,NaN,NaN,NaN,C0524910,NaN,D019698
6780099,ethnicity_disease,non-Hispanic Black,knee pain,ethnicity,disease,50.0,NaN,NaN,NaN,C0877149,NaN,D046788


In [25]:
# save to a csv file
kg_final1.to_csv('kg(3in1).csv', index=False)

In [26]:
r = kg_final['relation'].unique()
r

array(['subordinate1', 'subordinate0', 'gene_gene', 'drug_gene',
       'drug_disease', 'drug_drug', 'gene_effect/phenotype',
       'effect/phenotype_effect/phenotype', 'disease_effect/phenotype',
       'gene_disease', 'disease_disease', 'drug_effect/phenotype',
       'biological_process_biological_process',
       'molecular_function_molecular_function',
       'cellular_component_cellular_component', 'gene_molecular_function',
       'gene_cellular_component', 'gene_biological_process',
       'exposure_gene', 'exposure_disease', 'exposure_exposure',
       'exposure_biological_process', 'exposure_molecular_function',
       'exposure_cellular_component', 'pathway_pathway', 'gene_pathway',
       'anatomy_anatomy', 'gene_anatomy', 'drug_sdoh', 'disease_sdoh',
       'age_disease', 'age_drug', 'ethnicity_drug', 'ethnicity_disease',
       'religion_drug', 'religion_disease', 'marital_status_disease',
       'marital_status_drug', 'gender_drug', 'gender_disease',
       'insurance_d

In [27]:
len(r)

50

In [35]:
mimicKG

,head,relation,tail,weight,head_type,tail_type
0,*NF* Bicalutamide,drug_sdoh,admission,16,drug,SDoH
1,*NF* Bicalutamide,drug_sdoh,appointment,16,drug,SDoH
2,*NF* Bicalutamide,drug_sdoh,aricept 10 mg,16,drug,SDoH
3,*NF* Bicalutamide,drug_sdoh,assisted care facility,16,drug,SDoH
4,*NF* Bicalutamide,drug_sdoh,bicalutamide,16,drug,SDoH
...,...,...,...,...,...,...
923384,widowed,marital_status_drug,Vitamin D,60,marital_status,drug
923385,widowed,marital_status_drug,Warfarin,41,marital_status,drug
923386,widowed,marital_status_drug,Zolpidem Tartrate,12,marital_status,drug
923387,widowed,marital_status_drug,traZODONE,18,marital_status,drug


In [36]:
t1 = mimicKG[['head', 'head_type']]
t2 = mimicKG[['tail', 'tail_type']]
t2.columns = ['head', 'head_type']
# concat pkg1 and pkg2
testid = pd.concat([t1, t2], axis=0)
testid = testid.drop_duplicates(subset=('head', 'head_type'))
testid['x_index'] = testid.groupby('head_type').cumcount().astype(int)
testid.reset_index(drop=True, inplace=True)
testid

,head,head_type,x_index
0,*NF* Bicalutamide,drug,0
1,*NF* Budesonide,drug,1
2,*NF* Ertapenem Sodium,drug,2
3,*NF* Imatinib Mesylate,drug,3
4,0.45 % Sodium Chloride,drug,4
...,...,...,...
7536,daily treatment plan,SDoH,5103
7537,eval ct scan,SDoH,5104
7538,mental status medications,SDoH,5105
7539,rehab 2 status,SDoH,5106


In [38]:
# get the rows with 'head_type' = 'disease'
testid[testid['head_type'] == 'disease']
  

,head,head_type,x_index
914,00845,disease,0
915,0088,disease,1
916,0090,disease,2
917,0093,disease,3
918,03285,disease,4
...,...,...,...
2391,V8542,disease,1458
2392,V8741,disease,1459
2393,V8801,disease,1460
2395,"['82021', '5856', '42832', '40391', '4160', '5...",disease,1461
